In [1]:
# Compute the CPRS for the ICPAC region forecasts

import numpy as np
import netCDF4 as nc
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import colors  # For consistency with Harris et. al 2022
from datetime import datetime, timedelta
import numpy.ma as ma
from scipy.stats import multivariate_normal
rng = np.random.default_rng()

In [2]:
# Default function to returns the weights used to score two points in an image.
# This function sets how important neighbouring points are considered to be in variogram_score.
#
# In the estimate of the expectation values, the difference between two points often increases with
# distance because their correlation decreases. In the estimate of the expectation values, the
# difference between two points often increases with distance because their correlation decreases.
# Downweighting pairs that are expected to have relatively weak correlations can therefore benifit
# the signal-to-noise ratio.
#
# Arguments:
#   x1 - Integer horizontal coordinate of point 1 in pixels.
#   y1 - Integer vertical   coordinate of point 1 in pixels.
#   x2 - Integer horizontal coordinate of point 2 in pixels.
#   y2 - Integer vertical   coordinate of point 2 in pixels.
# Returns:
#   weight  - The weight for the variogram_score for this pair of pixels.
#
def weights_function(x1, y1, x2, y2):
    
    # Exponential decay constant in units of pixels
    decay_constant = -1
    
    distance = np.sqrt((x1-x2)**2 + (y1-y2)**2)
    weight = np.exp(decay_constant * distance)
    return weight

# Return unit weights for all point combinations
def unit_weights(x1, y1, x2, y2):
    return 1.0

# Computes a variogram score image for ne member ensemble forecasts of measurement images of size nx by ny
# pixels averaged over nt images.
# Arguments:
#   measurements_images                 - An array of size (nt, nx, ny) or (nx, ny) or (ny) measurements.
#   forecast_ensemble_images            - An array of ensemble forecasts of the measurements of size
#                                         (nt, ne, nx, ny) or (ne, nx, ny) or (ne, ny) or (ny).
#   p = 0.5                             - The moment parameter of the score. 
#   weights_function = weights_function - A user defined function to return the weights for arbitrary pairs of
#                                         pixels. See the definition of the default weights_function above.
#   D_max = 5                           - The maximum distance in pixels at which to compare points in the score.
# Returns:
#   The contribution to the variogram score for each point. Has the same dimensions (nx,ny) as a single 
#   measurements image. To get the variogram score, average over each pixel.
#
def variogram_score_slow(measurements_images,
                    forecast_ensemble_images,
                    p=0.5,
                    weights_function=weights_function,
                    D_max=5):
    
    # Extract nt, ne, nx, ny and do basic parameter checks
    
    if (forecast_ensemble_images.ndim == 1):
        # A single vector
        nt = 1
        ne = 1
        nx = 1
        (ny) = forecast_ensemble_images.shape
        
        ntm = 1
        nxm = 1
        (nym) = measurements_images.shape
    
    elif (forecast_ensemble_images.ndim == 2):
        # An ensemble of vectors
        nt = 1
        nx = 1
        (ne,ny) = forecast_ensemble_images.shape
        
        ntm = 1
        nxm = 1
        (nym) = measurements_images.shape
    
    elif (forecast_ensemble_images.ndim == 3):
        # An ensemble of images
        nt = 1
        (ne, nx, ny) = forecast_ensemble_images.shape
        
        ntm = 1
        (nxm,nym) = measurements_images.shape
        
    elif (forecast_ensemble_images.ndim == 4):
        # nt image ensembles
        (nt, ne, nx, ny) = forecast_ensemble_images.shape
        
        (ntm,nxm,nym) = measurements_images.shape
        
    else:
        raise Exception(f"forecast_ensemble_images must be an array with 1, 2, 3 or 4 dimensions but it has {forecast_ensemble_images.ndim}.")
    
    if (nt != ntm):
        raise Exception(f"Number of forecasts ({nt}) does not equal the number of measurement fields ({ntm}).")
    if (nx != nxm):
        raise Exception(f"Number of forecast pixels (nx = {nx}) does not equal the number of measurement pixels (nx = {nxm}).")
    if (ny != nym):
        raise Exception(f"Number of forecast pixels (ny = {ny}) does not equal the number of measurement pixels (ny = {nym}).")
    if (nx*ny < 2):
        raise Exception(f"At least two variables are required for the variogram score but nx*ny = {nx*ny}.")
    if (p <= 0.0):
        raise Exception("p must be > 0")
    if (D_max < 1):
        raise Exception("D_max must be >= 1")
    
    if (forecast_ensemble_images.ndim < 4):
        # Make arrays a standard shape (RAM inefficient)
        forecast_ensemble_images = np.reshape(forecast_ensemble_images, (nt,ne,nx,ny))
        measurements_images = np.reshape(measurements_images, (ntm, nxm, nym))
    
    # The variogram score image
    score = np.zeros((nx, ny))
    count = np.zeros((nx, ny), dtype=int)
    
    # Where there are no measurements at all, mask the score
    mask = np.sum(np.isnan(measurements_images), axis=0) != nt
    
    # For each measurement time
    for k in range(nt):
    
        # For each point in the image
        for i1 in range(nx):
            for j1 in range(ny):
                
                # No point computing anything for this measurement if it is np.nan
                if not np.isnan(measurements_images[k,i1,j1]):
                    
                    # Loop over the points surrounding this one
                    pixel_score = 0  # The current score at this pixel
                    score_invalid = False  # The score is valid so far
                    for i2 in range(np.max([0,i1-D_max]), np.min([nx,i1+D_max+1])):
                        for j2 in range(np.max([0,j1-D_max]), np.min([ny,j1+D_max+1])):
                            
                            # If the point is within the cut-off radius and is not the point we are considering
                            # and it is not masked out for all time.
                            dist = np.sqrt((i1-i2)**2 + (j1-j2)**2)
                            if (dist <= D_max) and (dist > 0.0) and (mask[i2,j2]):

                                # Can only be here if the mask[i2,j2] is True, so this is a temporary missing measurement
                                if np.isnan(measurements_images[k,i2,j2]):
                                    score_invalid = True
                                    break
                                
                                # Estimate expectation values
                                E = np.mean(np.abs(forecast_ensemble_images[k,:,i1,j1] - forecast_ensemble_images[k,:,i2,j2])**0.5)

                                # Compute the measurement difference
                                dy = np.abs(measurements_images[k,i1,j1] - measurements_images[k,i2,j2])**p

                                # Compute the weights
                                w = weights_function(i1,j1,i2,j2)

                                # Compute the score for this pixel
                                pixel_score += w * (dy - E)**2
                                
                        if score_invalid:
                            break
                    
                    if not score_invalid:
                        score[i1,j1] += pixel_score
                        # Keep track of the number of measurements used at this pixel
                        count[i1,j1] += 1
    
    # Average over the number of measurement times
    score /= (count + (count == 0))  # Take into account counts of 0
    
    # Don't return a score when no score was computed
    score[count == 0] = np.nan
    
    return score

# More optimised version of variogram_score_slow above
def variogram_score(measurements_images,
                    forecast_ensemble_images,
                    p=0.5,
                    weights_function=weights_function,
                    D_max=5):
    
    # Extract nt, ne, nx, ny and do basic parameter checks
    
    if (forecast_ensemble_images.ndim == 1):
        # A single vector
        nt = 1
        ne = 1
        nx = 1
        (ny) = forecast_ensemble_images.shape
        
        ntm = 1
        nxm = 1
        (nym) = measurements_images.shape
    
    elif (forecast_ensemble_images.ndim == 2):
        # An ensemble of vectors
        nt = 1
        nx = 1
        (ne,ny) = forecast_ensemble_images.shape
        
        ntm = 1
        nxm = 1
        (nym) = measurements_images.shape
    
    elif (forecast_ensemble_images.ndim == 3):
        # An ensemble of images
        nt = 1
        (ne, nx, ny) = forecast_ensemble_images.shape
        
        ntm = 1
        (nxm,nym) = measurements_images.shape
        
    elif (forecast_ensemble_images.ndim == 4):
        # nt image ensembles
        (nt, ne, nx, ny) = forecast_ensemble_images.shape
        
        (ntm,nxm,nym) = measurements_images.shape
        
    else:
        raise Exception(f"forecast_ensemble_images must be an array with 1, 2, 3 or 4 dimensions but it has {forecast_ensemble_images.ndim}.")
    
    if (nt != ntm):
        raise Exception(f"Number of forecasts ({nt}) does not equal the number of measurement fields ({ntm}).")
    if (nx != nxm):
        raise Exception(f"Number of forecast pixels (nx = {nx}) does not equal the number of measurement pixels (nx = {nxm}).")
    if (ny != nym):
        raise Exception(f"Number of forecast pixels (ny = {ny}) does not equal the number of measurement pixels (ny = {nym}).")
    if (nx*ny < 2):
        raise Exception(f"At least two variables are required for the variogram score but nx*ny = {nx*ny}.")
    if (p <= 0.0):
        raise Exception("p must be > 0")
    if (D_max < 1):
        raise Exception("D_max must be >= 1")
    
    if (forecast_ensemble_images.ndim < 4):
        # Make arrays a standard shape (RAM inefficient)
        forecast_ensemble_images = np.reshape(forecast_ensemble_images, (nt,ne,nx,ny))
        measurements_images = np.reshape(measurements_images, (ntm, nxm, nym))
    
    # Precompute weights
    i1 = 0
    j1 = 0
    w = np.zeros((2*D_max+1,2*D_max+1))
    for i2 in range(i1-D_max, i1+D_max+1):
        for j2 in range(j1-D_max, j1+D_max+1):
            dist = np.sqrt((i1-i2)**2 + (j1-j2)**2)
            if (dist <= D_max) and (dist > 0.0):
                w[i2+D_max,j2+D_max] = weights_function(i1,j1,i2,j2)
            else:  # Out of range
                w[i2+D_max,j2+D_max] = 0
    
    # The variogram score image
    score = np.zeros((nx, ny))
    count = np.zeros((nx, ny), dtype=int)
    
    # Where there are no measurements at all, mask the score
    mask = np.sum(np.isnan(measurements_images), axis=0) != nt
    
    # For each measurement time
    for k in range(nt):
    
        # For each point in the image
        for i1 in range(nx):
            for j1 in range(ny):
                
                # No point computing anything for this measurement if it is np.nan
                if not np.isnan(measurements_images[k,i1,j1]):
                
                    i2_start = np.max([0,i1-D_max])
                    i2_end = np.min([nx,i1+D_max+1])

                    j2_start = np.max([0,j1-D_max])
                    j2_end = np.min([ny,j1+D_max+1])

                    # Compute the measurement difference
                    dy = np.abs(measurements_images[k,i1,j1] - measurements_images[k,i2_start:i2_end,j2_start:j2_end])**0.5
                    
                    # The weights for this specific i1 and j1
                    wij = w[i2_start-i1+D_max:i2_end-i1+D_max,j2_start-j1+D_max:j2_end-j1+D_max]
                    
                    # The valid numbers in dy are np.isnan(dy)                    
                    # The valid numbers in the mask are np.logical_not(mask[i2_start:i2_end,j2_start:j2_end])
                    # We don't care if the number is valid or not if the weight is zero
                    # If there is an extra np.nan then reject this measurement (XXX Ugly can I just use masked arrays?)
                    if (np.sum((wij > 0) & (np.isnan(dy) ^ np.logical_not(mask[i2_start:i2_end,j2_start:j2_end]))) == 0):
                        
                        # Estimate expectation values in the mask region
                        E = np.abs(forecast_ensemble_images[k,0,i1,j1] - forecast_ensemble_images[k,0,i2_start:i2_end,j2_start:j2_end])**0.5
                        for m in range(1,ne):
                            E += np.abs(forecast_ensemble_images[k,m,i1,j1] - forecast_ensemble_images[k,m,i2_start:i2_end,j2_start:j2_end])**0.5
                        E /= ne
                        
                        # Compute the score for this pixel
                        pixel_score = np.nansum(wij * (dy - E)**2)

                        score[i1,j1] += pixel_score
                        # Keep track of the number of measurements used at this pixel
                        count[i1,j1] += 1
                    
    # Average over the number of measurement times
    score /= (count + (count == 0))  # Take into account counts of 0
        
    # Don't return a score when no score was computed
    score[count == 0] = np.nan
    
    return score

In [3]:
year = 2020

IMERG_data_dir = "/home/c/cooperf/data/IMERG_6h"
# This is Andrew's quick botch on the data so don't trust it
IFS_data_dir = f"/home/c/cooperf/IFS/IFS-regICPAC-ens-tp"
plot_dir = "/home/c/cooperf/IFS/junk_testing_data/pictures"
save_plots = False

# To be consistent with the Harris et. al paper.
value_range_precip = (0.1, 15)

# Load the constant in time latitude and longitude from a random file
# file_name = IMERG_data_dir + "/3B-HHR.MS.MRG.3IMERG.20210930-S233000-E235959.1410.V06B.HDF5.nc4"
file_name = f"{IMERG_data_dir}/20200101_00.nc4"
nc_file = nc.Dataset(file_name)
latitude = np.array(nc_file["lat"][:])  # Cropping the first point for consistency with the IFS region
longitude = np.array(nc_file["lon"][:])
nc_file.close()

In [4]:
# Load example IMERG rainfall
d = datetime(year,1,13,0)
count = 360  # There is an additional number in the file name
# file_name = f"{IMERG_data_dir}/3B-HHR.MS.MRG.3IMERG.{year}{d.month:02d}{d.day:02d}-S{d.hour:02d}{d.minute:02d}00-E{d.hour:02d}{d.minute+29:02d}59.{count:04d}.V06B.HDF5.nc4"
file_name = f"{IMERG_data_dir}/{year}{d.month:02d}{d.day:02d}_{d.hour:02d}.nc4"
nc_file = nc.Dataset(file_name)
IMERG_rain = np.array(nc_file["precipitationCal"][:,:])
nc_file.close()

In [5]:
# Load example IFS rainfall with a 24 hour lead time
file_name = f"{IFS_data_dir}/{year}/tp_1.nc"
nc_file = nc.Dataset(file_name)
forecast_times = np.array(nc_file["time"][:])
valid_times = np.array(nc_file["valid_time"][:])
tp_IFS_accum = np.array(nc_file["tp"][12,:,4:6,1:,:])*1000/6  # Convert from m/6h to mm/h
tp_IFS = tp_IFS_accum[:,1,:,:] - tp_IFS_accum[:,0,:,:]
nc_file.close()

In [6]:
# Load example cGAN rainfall with a 24 hour lead time
file_name = f"/home/c/cooperf/data/cGAN/ICPAC/GAN_forecasts/GAN_12.nc"
nc_file = nc.Dataset(file_name)
forecast_times_GAN = np.array(nc_file["time"][:])
valid_times_GAN = np.array(nc_file["fcst_valid_time"][:])
tp_GAN = np.array(nc_file["precipitation"][0,:,0,:,:])
nc_file.close()

In [7]:
print(IMERG_rain.shape)
print(tp_IFS.shape)
print(tp_GAN.shape)

(384, 352)
(50, 384, 352)
(50, 384, 352)


In [8]:
IFS_variogram = variogram_score(IMERG_rain, tp_IFS)
cGAN_variogram = variogram_score(IMERG_rain, tp_GAN)

In [ ]:
# Now compare all of the GAN forecasts, IFS forecasts and IMERG data

# IFS_variogram_all = np.zeros((366,4,len(latitude),len(longitude)))
# cGAN_variogram_all = np.zeros((366,4,len(latitude),len(longitude)))
# forecast_times = np.zeros((366,4))
# valid_times = np.zeros((366,4))
# forecast_times_GAN = np.zeros((366,4))
# valid_times_GAN = np.zeros((366,4))

# XXX Broke so restarted
for day_num in range(43,366):   # 0 to 365 (366 days in 2020)
    print(day_num)
    for valid_time_num in range(4):  # There are 4 valid times in each GAN forecast

        # Load example IMERG rainfall
        d = datetime(year,1,2,6*valid_time_num) + timedelta(days=day_num)
        file_name = f"{IMERG_data_dir}/{year}{d.month:02d}{d.day:02d}_{d.hour:02d}.nc4"
        nc_file = nc.Dataset(file_name)
        IMERG_rain = np.array(nc_file["precipitationCal"][:,:])
        nc_file.close()
        
        # Load example IFS rainfall with a 24 hour lead time
        forecast_start_date = d-timedelta(days=1)  # 24 hour lead time
        file_name = f"{IFS_data_dir}/{year}/tp_{forecast_start_date.month}.nc"
        nc_file = nc.Dataset(file_name)
        forecast_time = np.array(nc_file["time"][forecast_start_date.day-1])
        valid_time = np.array(nc_file["valid_time"][forecast_start_date.day-1,4+valid_time_num])
        tp_IFS_accum = np.array(nc_file["tp"][forecast_start_date.day-1,:,4+valid_time_num:6+valid_time_num,1:,:])*1000/6  # Convert from m/6h to mm/h
        tp_IFS = tp_IFS_accum[:,1,:,:] - tp_IFS_accum[:,0,:,:]
        nc_file.close()
        
        # Load the cGAN forecast
        file_name = f"/home/c/cooperf/data/cGAN/ICPAC/GAN_forecasts/GAN_{day_num}.nc"
        nc_file = nc.Dataset(file_name)
        forecast_time_GAN = np.array(nc_file["time"][0])
        valid_time_GAN = np.array(nc_file["fcst_valid_time"][0,valid_time_num])
        tp_GAN = np.array(nc_file["precipitation"][0,:,valid_time_num,:,:])
        nc_file.close()

        # Compute the variogram score
        IFS_variogram = variogram_score(IMERG_rain, tp_IFS)
        cGAN_variogram = variogram_score(IMERG_rain, tp_GAN)
        
        # Save for later
        IFS_variogram_all[day_num,valid_time_num,:,:] = IFS_variogram
        cGAN_variogram_all[day_num,valid_time_num,:,:] = cGAN_variogram
        forecast_times[day_num,valid_time_num] = forecast_time
        valid_times[day_num,valid_time_num] = valid_time
        forecast_times_GAN[day_num,valid_time_num] = forecast_time_GAN
        valid_times_GAN[day_num,valid_time_num] = valid_time_GAN

43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


In [13]:
day_num

365

In [12]:
# Check the valid times are the same
print(np.max(np.abs(forecast_times - forecast_times_GAN)))
print(np.max(np.abs(valid_times - valid_times_GAN)))

0.0
0.0


In [14]:
# Save the data for later processing
output_data_dir = "/home/c/cooperf/IFS/To_ICPAC/Jan_2024"
file_name = "variogram_2020.nc"

# Create a new NetCDF file
rootgrp = nc.Dataset(f"{output_data_dir}/{file_name}", "w", format="NETCDF4")

# Describe where this data comes from
rootgrp.description = "IFS and cGAN variogram vs IMERG rain data for 2020."

# Create dimensions
longitude_dim = rootgrp.createDimension("longitude", len(longitude))
latitude_dim = rootgrp.createDimension("latitude", len(latitude))
time_dim = rootgrp.createDimension("time", None)
valid_time_dim = rootgrp.createDimension("valid_time", 4)
#ensemble_dim = rootgrp.createDimension("member", num_ensemble_members)

# Create the longitude variable
longitude_data = rootgrp.createVariable("longitude", "f4", ("longitude"), zlib=False)
longitude_data.units = "degrees_east"
longitude_data[:] = longitude   # Write the longitude data

# Create the latitude variable
latitude_data = rootgrp.createVariable("latitude", "f4", ("latitude"), zlib=False)
latitude_data.units = "degrees_north"
latitude_data[:] = latitude     # Write the latitude data

# Create the time variable
time_data = rootgrp.createVariable("time", "f4", ("time"), zlib=False)
time_data.units = "hours since 1900-01-01 00:00:00.0"
time_data[:] = forecast_times[:,0]     # Write the forecast_times data

# Create the valid_time variable
valid_time_data = rootgrp.createVariable("valid_time", "f4", ("time","valid_time"), zlib=False)
valid_time_data.units = "hours since 1900-01-01 00:00:00.0"
valid_time_data[:] = valid_times     # Write the valid_times data

# Create the IFS_CRPS variable
IFS_variogram_data = rootgrp.createVariable("IFS_variogram_score", "f4", ("time","valid_time","latitude","longitude"), zlib=True)
IFS_variogram_data.description = "IFS vs IMERG variogram score"
IFS_variogram_data.units = "mm/h"
IFS_variogram_data[:] = IFS_variogram_all     # Write the IMERG_rain_all data

# Create the IFS_CRPS variable
cGAN_variogram_data = rootgrp.createVariable("cGAN_variogram_score", "f4", ("time","valid_time","latitude","longitude"), zlib=True)
cGAN_variogram_data.description = "cGAN vs IMERG variogram score"
cGAN_variogram_data.units = "mm/h"
cGAN_variogram_data[:] = cGAN_variogram_all     # Write the IMERG_rain_all data

# Close the NetCDF file
rootgrp.close()

In [ ]:
IFS_variogram_all.shape

In [ ]:
np.mean(IFS_variogram_all[:,:,:,:])

In [ ]:
np.mean(cGAN_variogram_all[:,:,:,:])

In [ ]:
# Define the figure and each axis for the rows and columns
fig, axs = plt.subplots(nrows=1,ncols=2,
                        subplot_kw={'projection': ccrs.PlateCarree()},
                        figsize=(16,8))

# axs is a 2 dimensional array of `GeoAxes`. Flatten it into a 1-D array
axs=axs.flatten()

# Make the plots

ax=axs[0]
ax.set_facecolor('white')  # For consistency with Harris et. al 2022
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.RIVERS, linewidth=1,edgecolor='black')
ax.add_feature(cfeature.LAKES, linewidth=1,linestyle='-',edgecolor='black',facecolor='none')
c = ax.pcolormesh(longitude, latitude, np.mean(IFS_variogram_all[:,0,:,:], axis=0), 
                  # norm=colors.LogNorm(*value_range_precip),
                  vmin=0, vmax=0.7,
                  transform=ccrs.PlateCarree(), cmap='YlGnBu')
ax.set_title(f"IFS variogram",size=18)

ax=axs[1]
ax.set_facecolor('white')  # For consistency with Harris et. al 2022
ax.add_feature(cfeature.COASTLINE, linewidth=1)
ax.add_feature(cfeature.RIVERS, linewidth=1,edgecolor='black')
ax.add_feature(cfeature.LAKES, linewidth=1,linestyle='-',edgecolor='black',facecolor='none')
c = ax.pcolormesh(longitude, latitude, np.mean(cGAN_variogram_all[:,0,:,:], axis=0), 
                  # norm=colors.LogNorm(*value_range_precip),
                  vmin=0, vmax=0.7,
                  transform=ccrs.PlateCarree(), cmap='YlGnBu')
ax.set_title("cGAN variogram",size=18)

# Adjust the location of the subplots on the page to make room for the colorbar
fig.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.8,
                    wspace=0.02, hspace=0.02)

# Add a colorbar axis at the bottom of the graph
cbar_ax = fig.add_axes([0.81, 0.275, 0.0125, 0.45])

# Draw the colorbar
cb=fig.colorbar(c, cax=cbar_ax,orientation='vertical')
cb.ax.tick_params(labelsize=18)
cb.set_label('Rainfall (mm/h)',size=18)

# Save the picture
# plt.savefig(f"{plot_dir}/test.png", format="png", bbox_inches='tight')

In [ ]:
# So this is a case study
# Keep that for reporting

plt.pcolormesh(IFS_variogram,vmin=0,vmax=5)
plt.colorbar()
plt.title("IFS_variogram")
plt.show()

plt.pcolormesh(cGAN_variogram,vmin=0,vmax=5)
plt.colorbar()
plt.title("cGAN_variogram")
plt.show()

plt.pcolormesh(IMERG_rain,vmin=0,vmax=5)
plt.colorbar()
plt.title("IMERG_rain")
plt.show()

plt.pcolormesh(np.mean(tp_IFS,axis=0),vmin=0,vmax=5)
plt.colorbar()
plt.title("IFS rain mean")
plt.show()

plt.pcolormesh(np.mean(tp_GAN,axis=0),vmin=0,vmax=5)
plt.colorbar()
plt.title("cGAN rain mean")
plt.show()